In [6]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jun 25 16:20:12 2015
预处理

@author: Balázs Hidasi
"""

import numpy as np
import pandas as pd
import datetime as dt


chunkList=pd.read_csv("DataTimestampOrderByUser_Item_Time.csv",chunksize=1)

previousUser=0;#之前的用户
previousItem=0;#之前的物品
rePurchaseNum=0;    #交易次数
#一个Session的内容
aUsersChunk=[]
#测试集合数量
testSetNum=1;
chunks=[]
traningChunks=[]
testChunks=[]
sessionId=0

aUsersdf=[]
#上一个购买日期
preTime=0
#上一个Chunk
preChunk=None


for chunk in chunkList:
    tempdf=None
    #第一行第一个用户
    if(previousUser==0):
        #重复次数加1
        rePurchaseNum=rePurchaseNum+1 
        #购买日期
        preTime=chunk.Time.item()/3600/24;       
        chunk.Time =-1;
        aUsersChunk.append(chunk)
        
        previousUser=chunk.UserId.item()
        previousItem=chunk.ItemId.item()     
        continue    
    #同一个用户并且再次买了同一个商品
    if chunk.UserId.item()  == previousUser and chunk.ItemId.item() == previousItem:
        #重复次数加1
        rePurchaseNum=rePurchaseNum+1
        NowDay=chunk.Time.item()/3600/24
        purchaseCycle=NowDay-preTime
        #上一条设置purchasecycle
        aUsersChunk[-1].Time=purchaseCycle
        preTime=NowDay
        #chunk.Time =NowDay-preTime;
        chunk.Time =-1;
        aUsersChunk.append(chunk)
    #如同一用户但是不同商品
    elif chunk.UserId.item()  == previousUser and chunk.ItemId.item()  != previousItem:
        if rePurchaseNum>=3:#只取重复交易记录大于等于3的
            
            chunks.extend(aUsersChunk) 
            num=len(aUsersChunk)
            traningChunks.extend(aUsersChunk[:num-testSetNum-1])#分训练集
            testChunks.extend(aUsersChunk[num-testSetNum-1:-1])#分验证集 去掉最后一个没有purchasecycle的
            #aUsersChunk=[]
            aUsersChunk.clear()
        aUsersChunk.clear()
        rePurchaseNum=0
        rePurchaseNum=rePurchaseNum+1
        
        NowDay=chunk.Time.item()/3600/24; 
        preTime=NowDay
        chunk.Time =-1;
        aUsersChunk.append(chunk)        
    #新用户
    elif  chunk.UserId.item()  != previousUser:
        if rePurchaseNum>=3:#只取重复交易记录大于等于3的
            chunks.extend(aUsersChunk) 
            num=len(aUsersChunk)
            traningChunks.extend(aUsersChunk[:num-testSetNum-1])#分训练集
            testChunks.extend(aUsersChunk[num-testSetNum-1:-1])#分验证集
            #aUser                sChunk=[]
            aUsersChunk.clear()
        rePurchaseNum=0
        #aUsersChunk=[]
        aUsersChunk.clear()
        rePurchaseNum=rePurchaseNum+1
        
        NowDay=chunk.Time.item()/3600/24  
        preTime=NowDay
        chunk.Time =-1;
        aUsersChunk.append(chunk)
            
    previousUser=chunk.UserId.item()
    previousItem=chunk.ItemId.item()
#1.2.合并分块    
#df = pd.concat(chunks, ignore_index=True)
#print("traningChunks",traningChunks)
dftraning=pd.concat(traningChunks, ignore_index=True)
dftraning["SessionId"]=dftraning["UserId"].astype('str')+dftraning["ItemId"].astype('str')
dftest=pd.concat(testChunks, ignore_index=True)
    #1.3.只取event等于transaction的行
    #df=df[df.event=="transaction"]
    #1.4根据visitorid排序
    #df=df.sort_values(ascending=True,by="visitorid")
    
    
    
    #df.to_csv("dataOrderMoreThan10T.csv")
dftraning.to_csv("dataOrderMoreThanSame3TrainT2.csv")
dftest.to_csv("dataOrderMoreThanSame3TestT2.csv")
print("finish")

   
   


finish


In [5]:
list1=[0,1,2]
print(list1[:3-1-1])
print(list1[3-1-1:-1])

[0]
[1]
